
## Deploy model to Amazon SageMaker Endpoint

In [1]:
!python -m pip install --upgrade pip --quiet
!pip install -U awscli --quiet
!pip install -U sagemaker --quiet
!pip install pandas --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.33.5 requires botocore<1.34.0,>=1.33.5, but you have botocore 1.34.2 which is incompatible.
boto3 1.33.5 requires s3transfer<0.9.0,>=0.8.2, but you have s3transfer 0.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.2 requires botocore==1.34.2, but you have botocore 1.33.13 which is incompatible.
awscli 1.32.2 requires s3transfer<0.10.0,>=0.9.0, but you have s3transfer 0.8.2 which is incompatible.


In [2]:
import joblib
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.model import SKLearnModel
import boto3
import pandas as pd
import json
from sagemaker.s3 import S3Uploader

role = get_execution_role()
boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
region = boto_session.region_name
bucket = sm_session.default_bucket()
prefix = 'inference'

print(region)
print(role)
print(bucket)
print(prefix)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/dona/.config/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name Dona to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::871322254241:user/Dona, therefore it cannot be used as a SageMaker execution role

### Create model archive and source archive and upload them to S3

In [2]:
!rm -f model.tar.gz
!tar cvzf model.tar.gz -C ../model/ model.joblib -C code/ requirements.txt

model.joblib
requirements.txt


In [3]:
!rm -f sourcedir.tar.gz
!tar cvzf sourcedir.tar.gz --exclude='*ipynb*' -C code .

./
./requirements.txt
./inference.py


In [4]:
file_key = 'model.tar.gz'
model_artifact = S3Uploader.upload(file_key,'s3://{}/{}/model'.format(bucket, prefix))
print(model_artifact)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-871322254241/inference/model/model.tar.gz


In [5]:
file_key = 'sourcedir.tar.gz'
code_artifact = S3Uploader.upload(file_key,'s3://{}/{}/sourcedir'.format(bucket, prefix))
print(code_artifact)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-871322254241/inference/sourcedir/sourcedir.tar.gz


In [6]:
deploy_instance_type = 'ml.m4.xlarge'
endpoint_name = "predict-price"

In [7]:
# # Local mode deployment for testing purposes
# sklearn_model = SKLearnModel(model_data=model_artifact,
#                              role=role,
#                              entry_point="inference.py",
#                              source_dir=code_artifact,
#                              framework_version="1.2-1")

# predictor = sklearn_model.deploy(instance_type="local", initial_instance_count=1, endpoint_name=endpoint_name)

### Create SageMaker model with sklearn inference container

In [8]:
from sagemaker.image_uris import retrieve

container = retrieve('sklearn',
                        region=region,
                        version="1.2-1",
                        py_version='py3',
                        image_scope='inference')

In [9]:
model_name = "predict-price-model"

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_artifact,
        'Environment': {
            "SAGEMAKER_PROGRAM": "inference.py",
            "SAGEMAKER_SUBMIT_DIRECTORY": code_artifact,
        }
    },
)

### Real time hosted endpoint deployment and inference

In [10]:
endpoint_config_name = "predict-price-config"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": deploy_instance_type,
            "InitialInstanceCount": 1
        }
    ]
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

Created EndpointConfig: arn:aws:sagemaker:us-east-1:871322254241:endpoint-config/predict-price-config


In [11]:
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

In [ ]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

Waiting for endpoint to create...


### Invoke endpoint

In [13]:
df = pd.read_csv("Luxury watch.csv")
df

,Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price (USD)
0,Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.00,20.0,Black,Sapphire,Date,48 hours,"9,500"
1,Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800"
2,Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.00,20.0,White,Sapphire,Chronograph,42 hours,"4,200"
3,Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,"7,900"
4,Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,NaN,"2,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Breguet,Classique,18k Rose Gold,Leather,Automatic,30 meters,38.5,8.25,20.0,White,Sapphire,Date,38 hours,"21,500"
503,Blancpain,Fifty Fathoms,Stainless Steel,Fabric,Automatic,300 meters,45.0,15.50,23.0,Black,Sapphire,Date,120 hours,"13,500"
504,Longines,Master Collection,Stainless Steel,Leather,Automatic,30 meters,38.5,9.70,20.0,Blue,Sapphire,Date,64 hours,"1,800"
505,Vacheron Constantin,Overseas,Stainless Steel,Stainless Steel,Automatic,150 meters,41.5,11.00,22.0,Blue,Sapphire,Date,40 hours,"19,000"


In [14]:
targetcol = 'Price (USD)'
X = df.loc[:, ~df.columns.isin([targetcol])]

In [15]:
body = X.iloc[15].to_json()
body

'{"Brand":"Audemars Piguet","Model":"Royal Oak","Case Material":"Stainless Steel","Strap Material":"Stainless Steel","Movement Type":"Automatic","Water Resistance":"50 meters","Case Diameter (mm)":41.0,"Case Thickness (mm)":9.8,"Band Width (mm)":20.0,"Dial Color":"Silver","Crystal Material":"Sapphire","Complications":"Date","Power Reserve":"60 hours"}'

In [16]:
def send_request(endpoint_name, payload, sm_client):
    response = sm_client.invoke_endpoint(EndpointName=endpoint_name, ContentType="application/json", Body=payload)
    prediction = response['Body'].read()
    prediction = json.loads(prediction)["prediction"]
    return prediction

In [17]:
# # Local mode invocation
# local_sm_client = sagemaker.local.LocalSagemakerRuntimeClient()
# send_request(endpoint_name, body, local_sm_client)

In [18]:
send_request(endpoint_name, body, sm_runtime)

'22750.0'

### Cleanup

In [23]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '8fa5e306-6d96-485e-a9ea-e0c35aa1e121',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8fa5e306-6d96-485e-a9ea-e0c35aa1e121',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 14 Dec 2023 15:22:09 GMT'},
  'RetryAttempts': 0}}